In [1]:
import torch
import numpy as np

In [27]:
# institate testing vectors
size = 10000
mask_size = int(np.round(size * size * 0.01))
x = torch.rand((size, 1))
y = torch.rand((1, size))

In [3]:
def compute_mask(w, topk_percentage):
    threshold = np.quantile(w.reshape(-1), topk_percentage)
    mask = np.where(w >= threshold)
    return mask

In [29]:
m = x.mm(y)
mask = compute_mask(m, 0.99)

In [19]:
def masked_outer_product(x, y, mask):
    values = torch.zeros(len(mask[0]))
    x = x.reshape(-1)
    y = y.reshape(-1)
    for i in range(len(mask[0])):
        values[i] = x[mask[0][i]] * y[mask[1][0]]
    
    return torch.sparse_coo_tensor(mask, values, (len(x), len(y)))
        

In [30]:
%timeit masked_outer_product(x, y, mask)

8.89 s ± 97.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit m = x.mm(y)
# torch.sparse_coo_tensor(mask, m[mask], m.shape)

132 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
